<a href="https://colab.research.google.com/github/ghedin-alison/NLP/blob/main/Aula_3_6_IA_PLN_Exerc%C3%ADcio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carregando o dataset

---


In [93]:
import pandas as pd

df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.describe()

,nome,descricao,categoria
count,4080,2916,4080
unique,3696,2460,4
top,Boneco Dragon Ball Z Son Gokou,JOGO ORIGINAL. NOVO. LACRADO. PRONTA ENTREGA. ...,livro
freq,20,39,1020


In [94]:
# limpa nulos
df.dropna(inplace=True)

# concatena nome e descrição do produto
df["texto"] = df['nome'] + " " + df['descricao']

df.texto[0]

' O Hobbit - 7ª Ed. 2013  Produto NovoBilbo Bolseiro é um hobbit que leva uma vida confortável e sem ambições. Mas seu contentamento é perturbado quando Gandalf, o mago, e uma companhia de anões batem à sua porta e levam-no para uma expedição. Eles têm um plano para roubar o tesouro guardado por Smaug, o Magnífico, um grande e perigoso dragão. Bilbo reluta muito em participar da aventura, mas acaba surpreendendo até a si mesmo com sua esperteza e sua habilidade como ladrão!CaracterísticasAutor: Tolkien, J. R. R.Peso: 0.44I.S.B.N.: 9788578277109Altura: 20.000000Largura: 13.000000Profundidade: 1.000000Número de Páginas: 328Idioma: PortuguêsAcabamento: BrochuraNúmero da edição: 7Ano da edição: 2013'

In [95]:
df.describe()

,nome,descricao,categoria,texto
count,2916,2916,2916,2916
unique,2584,2460,4,2646
top,Boneco Dragon Ball Z Son Gokou,JOGO ORIGINAL. NOVO. LACRADO. PRONTA ENTREGA. ...,livro,Boneco Dragon Ball Z Son Gokou Produto novo
freq,20,39,838,20


Dividindo o dataframe em duas amostras: treino e teste

In [96]:
from sklearn.model_selection import train_test_split

# divisão da amostra entre treino e teste
df_train, df_test = train_test_split(
      df, 
      test_size = 0.3, 
      random_state = 42
  )

print('Treino:\n')
print(df_train.info())
print('\nTeste:\n')
print(df_test.info())

Treino:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2041 entries, 1799 to 1050
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nome       2041 non-null   object
 1   descricao  2041 non-null   object
 2   categoria  2041 non-null   object
 3   texto      2041 non-null   object
dtypes: object(4)
memory usage: 79.7+ KB
None

Teste:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 875 entries, 2700 to 2734
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nome       875 non-null    object
 1   descricao  875 non-null    object
 2   categoria  875 non-null    object
 3   texto      875 non-null    object
dtypes: object(4)
memory usage: 34.2+ KB
None


Tratamento e vetorização

## 1 - Contagem de termos com unigrama

In [102]:
# 1 - Contagem de termos com unigrama
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

stops = nltk.corpus.stopwords.words('portuguese')

#vect = CountVectorizer(ngram_range=(1,1))
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_train.texto)
X_train = vect.transform(df_train.texto)
y_train = df_train["categoria"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [103]:
# treinamento do modelo ávore de decisão com o dataframe de treino
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train) ## objeto que será salvo pra utilizar num pipeline de produção

# transforma o dataframe de teste em vetor (features)
X_test = vect.transform(df_test.texto)

# escoragem da classificação na amostra de teste
y_prediction = tree.predict(X_test)

# mensuração do resultado pela acurácia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(f"Acurácia com a primeira tentativa removendo stopwords:{accuracy}")

Acurácia com a primeira tentativa removendo stopwords:0.9611428571428572


In [72]:
# 2 - TF com unigrama - referencia
from sklearn.feature_extraction.text import TfidfVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l1')
#vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l1', stop_words=stops)
vect.fit(df_train.texto)
X_train = vect.transform(df_train.texto)
y_train = df_train["categoria"]

In [73]:
# treinamento do modelo ávore de decisão com o dataframe de treino
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train) ## objeto que será salvo pra utilizar num pipeline de produção

# transforma o dataframe de teste em vetor (features)
X_test = vect.transform(df_test.texto)

# escoragem da classificação na amostra de teste
y_prediction = tree.predict(X_test)

# mensuração do resultado pela acurácia
y_test = df_test["categoria"]
accuracy = accuracy_score(y_prediction, y_test)
print(f"Acurácia com a segunda tentativa removendo stopwords:{accuracy}")

Acurácia com a segunda tentativa removendo stopwords:0.944


In [83]:
# 3 - TF-IDF com unigrama e aplicando o stemização RSLP
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

df_train['tokens'] = df_train.texto.apply(word_tokenize)

from nltk.stem.rslp import RSLPStemmer
import nltk
nltk.download('rslp')

rslp = RSLPStemmer()

def stem_pandas(line):
  return ' '.join([rslp.stem(token) for token in line])

df_train['stemmer'] = df_train.tokens.apply(stem_pandas)
df_train.head(2)

,nome,descricao,categoria,texto,tokens,stemmer
1799,Refil De Dardos Nerf Zombie Strike 1 Pacote ...,PRODUTO ORIGINAL HASBROCÓDIGO DE REFERÊNCIA DO...,brinquedo,Refil De Dardos Nerf Zombie Strike 1 Pacote ...,"[Refil, De, Dardos, Nerf, Zombie, Strike, 1, P...",refil de dard nerf zombi strik 1 pacot com 30 ...
1339,Bandana Naruto Shippuden Konoha Cosplay Ninja...,Naruto Shippuden Bandana Konoha Cosplay Ninja ...,brinquedo,Bandana Naruto Shippuden Konoha Cosplay Ninja...,"[Bandana, Naruto, Shippuden, Konoha, Cosplay, ...",band narut shippuden konoh cosplay ninj var vi...


In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer

#stops = nltk.corpus.stopwords.words('portuguese')

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
#vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
#vect = TfidfVectorizer(ngram_range=(1,1), use_idf=False, stop_words=stops)
#vect = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=stops, smooth_idf=False, norm='l1')

vect.fit(df_train.stemmer)
text_vect_train = vect.transform(df_train.stemmer)

<2041x22334 sparse matrix of type '<class 'numpy.float64'>'
	with 212597 stored elements in Compressed Sparse Row format>

In [88]:
# Aplicando a transformação no df de teste
df_test['tokens'] = df_test.texto.apply(word_tokenize)
df_test['stemmer'] = df_test.tokens.apply(stem_pandas)
df_test.head(2)

,nome,descricao,categoria,texto,tokens,stemmer
2700,Estojo Duo Iluminador E Bronzer Belle Angel M...,DUO ILUMINADOR E BRONZER BELLE ANGEL - B025O D...,maquiagem,Estojo Duo Iluminador E Bronzer Belle Angel M...,"[Estojo, Duo, Iluminador, E, Bronzer, Belle, A...",estoj duo ilumin e bronz bell angel maqui em p...
1297,Patrulha Canina Carrinhos De Fricção 6 Person...,Kit carrinhos da Patrulha canina Tamanho: Cerc...,brinquedo,Patrulha Canina Carrinhos De Fricção 6 Person...,"[Patrulha, Canina, Carrinhos, De, Fricção, 6, ...",patrulh canin carr de fricç 6 person nov kit c...


Treinando e validando o modelo

In [89]:
x_train = text_vect_train
y_train = df_train["categoria"]

from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
tree.fit(x_train, y_train)

#valide qual vetorização está usando: com o texto original ou com stemmer
x_test = vect.transform(df_test.texto)
#x_test = vect.transform(df_test.stemmer)
y_test = df_test["categoria"]

#tree.score(x_train, y_train)
tree.score(x_test, y_test)

0.6251428571428571

Exemplo 1.1 - 0.9565714285714285

Exemplo 1.2 - 0.9611428571428572

Exemplo 2.1 - 0.944

Exemplo 2.2 - 0.952

Exemplo 3.1 - 0.9234285714285714

Exemplo 3.2 - 0.9222857142857143

Exemplo 3.3 - 0.9268571428571428

Exemplo 3.4 - 0.9234285714285714
<br>

**Melhor modelo foi do exemplo 1.2: 0.9611428571428572**

Aplicando a predição em um novo texto

In [99]:
from sklearn.metrics import accuracy_score

y_prediction = tree.predict(x_test)

accuracy = accuracy_score(y_prediction, y_test)

print(accuracy)

ValueError: ignored

In [104]:
texto = """
Maleta Exclusiva Ruby Rose Profissional Completa.
"""

texto_vetor = vect.transform([texto])

In [105]:
tree.predict(texto_vetor)

array(['maquiagem'], dtype=object)

## Pipeline completo de treino e teste

Carregando os dados

In [25]:
import pandas as pd

df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.describe()

,nome,descricao,categoria
count,4080,2916,4080
unique,3696,2460,4
top,Boneco Dragon Ball Z Son Gokou,JOGO ORIGINAL. NOVO. LACRADO. PRONTA ENTREGA. ...,livro
freq,20,39,1020


Normalização

In [26]:
# limpa nulos
df.dropna(inplace=True)

# concatena nome e descrição do produto
df["texto"] = df['nome'] + " " + df['descricao']

Amostra treino e teste antes da vetorização

In [27]:
from sklearn.model_selection import train_test_split

# divisão da amostra entre treino e teste
df_train, df_test = train_test_split(
      df, 
      test_size = 0.3, 
      random_state = 42
  )

Tratamento e vetorização

In [28]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
# 1 - Contagem de termos com unigrama
from sklearn.feature_extraction.text import CountVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_train.texto)
text_vect = vect.transform(df_train.texto)

Treinamento do modelo

In [30]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_vect, df_train["categoria"])

DecisionTreeClassifier(random_state=42)

Validação do modelo

In [31]:
x_test = vect.transform(df_test.texto)
y_test = df_test["categoria"]

tree.score(x_test, y_test)

0.9611428571428572